# C1-W1: Content Based Image Retrieval (Week 1)

## Team 7
- Marco Cordón
- Iñaki Lacunza
- Cristian Gutiérrez

### Summary of the methods we have applied:
- **Method 1**: CieLAB + L1 Distance + 64 bins
- **Method 2**: RGB + Hellinger Kernel + 8 bins

In [1]:
import os
import skimage
import matplotlib.pyplot as plt
import numpy as np

import pandas as pd

import cv2
import pickle

### Task 1: Create Museum and query image descriptors

<ins>Image Descriptors used</ins>: 
- **Method 1**: CieLAB concatenated Histograms
- **Method 2**: 3D RGB concatenated Histograms

The Histograms are normalized.

In [2]:
def get_dict_from_data(path, extension=".jpg"):
    dictionary = dict()
    for file in sorted(os.listdir(path)):
        if file.endswith(extension):
            key = os.path.splitext(file)[0].split('_').pop()
            dictionary[key] = cv2.imread(os.path.join(path, file))
            
    return dictionary

In [3]:
def histogram(data, n_channels = 3, color = "RGB", mask = None):
    
    hist_images = dict()
    
    for k, v in data.items():
        # Get image shape for normalization
        h, w, c = v.shape
        #print(f"Shape of the {k} image: height->{h}, width->{w}, number of channels->{c}")
        num_values = h*w*c
        
        # Histogram calculation
        
        if color == "RGB":
            
            if mask: 
                red_color = cv2.calcHist([v], [1], mask[k], [8], [0, 256])
            else:
                red_color = cv2.calcHist([v], [1], None, [8], [0, 256])
                
            
            if mask: 
                blue_color = cv2.calcHist([v], [0], mask[k], [8], [0, 256])
            else:
                blue_color = cv2.calcHist([v], [0], None, [8], [0, 256])
            
            if mask: 
                green_color = cv2.calcHist([v], [2], mask[k], [8], [0, 256])
            else:
                green_color = cv2.calcHist([v], [2], None, [8], [0, 256])
                
               
            histogram = np.concatenate((red_color, green_color, blue_color))
            histogram /= np.sum(histogram)
            
            
        elif color == "Lab":
            
            v = cv2.cvtColor(v, cv2.COLOR_BGR2Lab)
            
            if mask: 
                a = cv2.calcHist([v], [1], mask[k], [64], [0, 256])
            else:
                a = cv2.calcHist([v], [1], None, [64], [0, 256])
                            
            if mask: 
                L = cv2.calcHist([v], [0], mask[k], [64], [0, 256])
            else:
                L = cv2.calcHist([v], [0], None, [64], [0, 256])
                
            
            if mask: 
                b = cv2.calcHist([v], [2], mask[k], [64], [0, 256])
            else:
                b = cv2.calcHist([v], [2], None, [64], [0, 256])
                 
            histogram = np.concatenate((L, a, b))
            histogram /= np.sum(histogram)           
            
        elif color == "HSV":
            hsv = cv2.cvtColor(v, cv2.COLOR_BGR2HSV)
            
            if mask: 
                s = cv2.calcHist([v], [1], mask[k], [64], [0, 256])
            else:
                s = cv2.calcHist([v], [1], None, [64], [0, 256])
                
            
            if mask: 
                h = cv2.calcHist([v], [0], mask[k], [64], [0, 256])
            else:
                h = cv2.calcHist([v], [0], None, [64], [0, 256])
                
            
            if mask: 
                v = cv2.calcHist([v], [2], mask[k], [64], [0, 256])
            else:
                v = cv2.calcHist([v], [2], None, [64], [0, 256])
                        
            histogram = np.concatenate((h, s, v))
            histogram /= np.sum(histogram)
            
        hist_images[k] = histogram.flatten()
    
    return hist_images

In [4]:
def plot_histogram(hist, channels = 3, color="rgb", size = 256):
    plt.figure(figsize = (15, 15))
    
    if color == "rgb": 
        colors = "rgb"
    else:
        colors=["black", "yellow", "brown"]
    
    if channels == 3:
        fig, axs = plt.subplots(nrows = 1, ncols = channels)
        
        fig.tight_layout(pad=3.0)
        
        axs[0].plot(hist[: size], c = colors[0])
        axs[0].set_title("Channel 1")
        if size == 256: axs[1].set_xticks(range(0, 256, 100))
        else: axs[2].set_xticks(range(0, size, 4))
        
        axs[1].plot(hist[size : size*2], c = colors[1])
        axs[1].set_title("Channel 2")
        if size == 256: axs[1].set_xticks(range(0, 256, 100))
        else: axs[2].set_xticks(range(0, size, 4))
        
        axs[2].plot(hist[size*2 :], c = colors[2])
        axs[2].set_title("Channel 3")
        if size == 256: axs[2].set_xticks(range(0, 256, 100))
        else: axs[2].set_xticks(range(0, size, 4))
        
    elif channels == 1:
        plt.plot(hist, c = "black")
        

### Task 2: Similarity measures

<ins>Metrics we are using:</ins>
- **Method 1**: L1 Distance (distance metric)
- **Method 2**: Hellinger Kernel (similarity metric)

We have chosen the metrics for each method based on a simple benchmark performed on the query sets (slides).

In [5]:
def Euclidean(h1, h2):
    return np.linalg.norm(h1 - h2)

In [6]:
def L1_distance(h1, h2):
    result = np.subtract(h1, h2)
    result = np.absolute(result)
    return np.sum(result)

In [7]:
def XSquaredDistance(h1, h2):
    result = ((h1 - h2)**2 / (h1 + h2))
    result = np.nan_to_num(result, nan = 0.0)
    return np.sum(result)

In [8]:
def HistogramIntersection(h1, h2):
    return np.sum(np.minimum(h1, h2))

In [9]:
def HellingerKernel(h1, h2):
    x = np.multiply(h1, h2)
    x = np.sqrt(x)
    return np.sum(x)

In [10]:
def compare(query, database, method):
    result = dict()
    for k, v in database.items():
        if method == "Euclidean": result[k] = Euclidean(v, query)
        elif method == "L1": result[k] = L1_distance(v, query)
        elif method == "X_Squared": result[k] = XSquaredDistance(v, query)
        elif method == "Histogram_Intersection": result[k] = HistogramIntersection(v, query)
        elif method == "Hellinger_Kernel": result[k] = HellingerKernel(v, query)
        
    return result

##### Obtaining ground truth indexes:

In [11]:
with open('./qsd1_w1/gt_corresps.pkl', 'rb') as f:
    cor1 = pickle.load(f)

with open('./qsd2_w1/gt_corresps.pkl', 'rb') as f:
    cor2 = pickle.load(f)

### Task 3: Retrieval System

In [12]:
def k_neighbours(dictionary, k=10, rev=False):
    result_dict = dict(sorted(dictionary.items(), key=lambda item: item[1], reverse=rev))
    return [int(keys) for keys,v in result_dict.items()][:k]

### Task 4: Evaluation using map@k

We are using the ml_metrics functions locally.

In [13]:
def apk(actual, predicted, k=10):
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [14]:
def calculateMAPK(histograms, histograms_bbdd, method, ground_truth = None, topK=10, rev = False):
    apk_list = []
              
    for k, v in histograms.items():
        apk_list += [k_neighbours(compare(v, histograms_bbdd, method), topK, rev = rev)]
      
    if ground_truth:
        #print(ground_truth)
        #print(apk_list)
        return mapk(actual=ground_truth, predicted=apk_list, k=topK)
    else:
        return apk_list
    

### Task 5: Background removal using color

We are using an **Otsu threshold** + **Morphological operations**.

More precisely the following steps:
1) Gaussian blur

2) Otsu Threshold

3) Normalization

4) Morphological operations:

    4.1) Gradient
    
    4.2) Or (Gradient + Otsu Threshold)
    
    4.3) 2x Closing
    
    4.4) Opening
    
    4.5) Closing
    
    4.6) Erode
    


The kernel size used for the morphological operations is relative to the image size.

In [15]:
def get_mask_with_percentages(img, path=None):
    # 1.
    blur_img = cv2.GaussianBlur(img[:,:,0], (25,25), 0)
            
    # 2. 
    ret2, th2 = cv2.threshold(blur_img, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    
    # 3.
    th2 = th2 / 255.0
    
    # Calculting the kernel size relative to the image size
    h, w, c = img.shape
    
    hper = 0.05
    wper = 0.05
    
    hker = int(h * hper)
    wker = int(w * wper)
    
    # 4.
    # Gradient
    kernel = np.ones((hker, wker),np.uint8)
    gradient = cv2.morphologyEx(th2, cv2.MORPH_GRADIENT, kernel, cv2.BORDER_CONSTANT, borderValue=1)
    
    # We need to invert the mask
    th2 = 1 - th2
    
    # Or
    mask = (th2 == 1) + (gradient == 1)
    mask = (255* mask).astype(np.uint8)
    
    # 2x Closing
    kernel_closing1 = np.ones((int(hker*0.5), int(wker*0.5)), np.uint8)
    closing1 = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel_closing1)
    kernel_closing2 = np.ones((int(hker*1.), int(wker*1.)), np.uint8)
    closing2 = cv2.morphologyEx(closing1, cv2.MORPH_CLOSE, kernel_closing2)
    
    # Opening:
    kernel_opening = np.ones((int(hker),int(wker)),np.uint8)
    opening = cv2.morphologyEx(closing2, cv2.MORPH_OPEN, kernel_opening)
    
    # 3.rd Closing
    kernel_closing3 = np.ones((int(hker*1.25), int(wker*1.25)),np.uint8)
    closing3 = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel_closing3)
    
    # Erosion
    kernel_erode = np.ones((int(hker*1.), int(wker*1.)),np.uint8)
    erode = cv2.erode(closing3, kernel_erode)        
    
    # Preprocessing steps visualization:
    """
    plt.axis("off")
    fig, ax = plt.subplots(nrows=3, ncols=3)
    
    ax[0, 0].imshow(img)
    ax[0, 1].imshow(th2, cmap = "gray") # blur
    ax[0, 2].imshow(gradient, cmap = "gray") # gradient
    ax[1, 0].imshow(mask, cmap = "gray") # or
    ax[1, 1].imshow(closing1, cmap = "gray") # closing1
    ax[1, 2].imshow(closing2, cmap = "gray") # closing2
    ax[2, 0].imshow(opening, cmap = "gray") # opening
    ax[2, 1].imshow(closing3, cmap = "gray") # closing3
    ax[2, 1].imshow(erode, cmap = "gray") # erode
    
    """

    return erode
    

### Task 6: Evaluation of picture masks and retrieval system

In [16]:
def get_masks_and_comparisons(query_set, path = None):
    correct_masks = get_dict_from_data(path=QSD2_PATH, extension=".png")
    
    all_results = dict()
    masks = dict()
    
    for name, mask in correct_masks.items():
        mask = cv2.cvtColor(mask ,cv2.COLOR_BGR2GRAY)
        predicted_mask = get_mask_with_percentages(query_set[name])
        
        # If path is True, save masks in a specified path. For saving test masks
        if path:
            if not os.path.exists(path):
                os.makedirs(path)
                
            filename = os.path.join(path, name + ".png")
            cv2.imwrite(filename, predicted_mask)
        
        # Compute metrics
        tp  = ((mask == 255) & (predicted_mask == 255)).sum()
        
        fn = ((mask == 255) & (predicted_mask == 0)).sum()
        
        fp = ((mask == 0) & (predicted_mask == 255)).sum()
        
        tn = ((mask == 0) & (predicted_mask == 0)).sum()
        
        #print(f"True Positives: {tp}  ,  False Negatives: {fn}  , False Positives: {fp}  ,  True Negatives: {tn}")
        #print(f"Total number of pixels: {tp + fn + fp + tn}")
        
        # Compute precision, recall and F1
        precision = tp / (tp + fp)
        recall = tp / (tp + fn)
        f1 = 2 * precision * recall / (precision + recall) 
        #print(f"Precision: {precision} , Recall: {recall} , F1: {f1}")
        
        #print()
        #print()
        
        
        result = {"precision": precision, "recall": recall, "f1": f1}
        all_results[name] = result
        masks[name] = predicted_mask
        
    dataframe = pd.DataFrame.from_dict(all_results)

    # Return results as dictionary and dataframe for comparing
    # And return the computed masks to apply them on the second query set
    return all_results, dataframe, masks

In [17]:
def get_masks(query_set, path = None):
    
    masks = dict()
    
    for name, img in query_set.items():
        #mask = cv2.cvtColor(mask ,cv2.COLOR_BGR2GRAY)
        predicted_mask = get_mask_with_percentages(query_set[name])
        
        # If path is True, save masks in a specified path. For saving test masks
        if path:
            if not os.path.exists(path):
                os.makedirs(path)
                
            filename = os.path.join(path, name + ".png")
            cv2.imwrite(filename, predicted_mask)
            
        masks[name] = predicted_mask
        
    return masks
        
        

In [18]:
def apply_mask(querys, masks):
    
    masked_qs = dict()
    
    for name, img in querys.items():
        img_mask = masks[name]
        
        masked_image = cv2.bitwise_and(img, img, mask = img_mask.astype(np.uint8))
        masked_qs[name] = masked_image

    return masked_qs

# Task 1-4

## Method1: CIELab, L1 distance, 64 bins

In [19]:
BBDD_PATH = './BBDD/'
QSD1_PATH = './qsd1_w1/'
QSD2_PATH = './qsd2_w1/'

bbdd = get_dict_from_data(BBDD_PATH)
qsd1 = get_dict_from_data(QSD1_PATH)
qsd2 = get_dict_from_data(QSD2_PATH)

histograms_bbdd_Lab = histogram(bbdd, color = "Lab")
histograms_qsd1_Lab = histogram(qsd1, color = "Lab")
histograms_qsd2_Lab = histogram(qsd2, color = "Lab")


with open('./qsd1_w1/gt_corresps.pkl', 'rb') as f:
    cor1 = pickle.load(f)

with open('./qsd2_w1/gt_corresps.pkl', 'rb') as f:
    cor2 = pickle.load(f)

    
MAPK_QS1_Lab_1 = calculateMAPK(histograms_qsd1_Lab, histograms_bbdd_Lab, "L1", cor1, 1, rev = False)
MAPK_QS1_Lab_5 = calculateMAPK(histograms_qsd1_Lab, histograms_bbdd_Lab, "L1", cor1, 5, rev = False)

print(f"Query Set 1 Lab results:\n k=1 -> {MAPK_QS1_Lab_1} \n k=5 -> {MAPK_QS1_Lab_5} \n\n")

MAPK_QS2_Lab_1_no_mask = calculateMAPK(histograms_qsd2_Lab, histograms_bbdd_Lab, "L1", cor2, 1, rev = False)
MAPK_QS2_Lab_5_no_mask = calculateMAPK(histograms_qsd2_Lab, histograms_bbdd_Lab, "L1", cor2, 5, rev = False)

print(f"Query Set 2 Lab results no masks:\n k=1 -> {MAPK_QS2_Lab_1_no_mask} \n k=5 -> {MAPK_QS2_Lab_5_no_mask} \n\n")

Query Set 1 Lab results:
 k=1 -> 0.5333333333333333 
 k=5 -> 0.5994444444444444 


Query Set 2 Lab results no masks:
 k=1 -> 0.13333333333333333 
 k=5 -> 0.14 




## Method 2: RGB, HellingerKernel, 8 bins

In [20]:
BBDD_PATH = './BBDD/'
QSD1_PATH = './qsd1_w1/'
QSD2_PATH = './qsd2_w1/'

bbdd = get_dict_from_data(BBDD_PATH)
qsd1 = get_dict_from_data(QSD1_PATH)
qsd2 = get_dict_from_data(QSD2_PATH)

histograms_bbdd_RGB = histogram(bbdd, color = "RGB")
histograms_qsd1_RGB = histogram(qsd1, color = "RGB")
histograms_qsd2_RGB = histogram(qsd2, color = "RGB")


with open('./qsd1_w1/gt_corresps.pkl', 'rb') as f:
    cor1 = pickle.load(f)

with open('./qsd2_w1/gt_corresps.pkl', 'rb') as f:
    cor2 = pickle.load(f)

    
MAPK_QS1_RGB_1 = calculateMAPK(histograms_qsd1_RGB, histograms_bbdd_RGB, "Hellinger_Kernel", cor1, 1, rev = True)
MAPK_QS1_RGB_5 = calculateMAPK(histograms_qsd1_RGB, histograms_bbdd_RGB, "Hellinger_Kernel", cor1, 5, rev = True)
 
print(f"Query Set 1 Method 2 results:\n k=1 -> {MAPK_QS1_RGB_1} \n k=5 -> {MAPK_QS1_RGB_5} \n\n")

MAPK_QS2_RGB_1_no_mask = calculateMAPK(histograms_qsd2_RGB, histograms_bbdd_RGB, "Hellinger_Kernel", cor2, 1, rev = True)
MAPK_QS2_RGB_5_no_mask = calculateMAPK(histograms_qsd2_RGB, histograms_bbdd_RGB, "Hellinger_Kernel", cor2, 5, rev = True)

print(f"Query Set 2 Method 2 results no masks:\n k=1 -> {MAPK_QS2_RGB_1_no_mask} \n k=5 -> {MAPK_QS2_RGB_5_no_mask} \n\n")

Query Set 1 Method 2 results:
 k=1 -> 0.43333333333333335 
 k=5 -> 0.45 


Query Set 2 Method 2 results no masks:
 k=1 -> 0.0 
 k=5 -> 0.04 




# Task 5-6

## Method 1: CIELab, L1 distance, 64 bins

In [21]:
BBDD_PATH = './BBDD/'
QSD2_PATH = './qsd2_w1/'

bbdd = get_dict_from_data(BBDD_PATH)
qsd2 = get_dict_from_data(QSD2_PATH)

histograms_bbdd_Lab = histogram(bbdd, color = "Lab")

all_results, dataframe, masks = get_masks_and_comparisons(qsd2)

dataframe_mean = dataframe.mean(axis=1)
print(f"{dataframe_mean} \n\n")

masked_histograms_Lab = histogram(qsd2, mask = masks, color="Lab")

MAPK_QS2_Lab_1_mask = calculateMAPK(masked_histograms_Lab, histograms_bbdd_Lab, "L1", cor2, 1, rev = False)
MAPK_QS2_Lab_5_mask = calculateMAPK(masked_histograms_Lab, histograms_bbdd_Lab, "L1", cor2, 5, rev = False)

print(f"Query Set 2 Method 1 results with masks:\n k=1 -> {MAPK_QS2_Lab_1_mask} \n k=5 -> {MAPK_QS2_Lab_5_mask} \n\n")

precision    0.836096
recall       0.916969
f1           0.868773
dtype: float64 


Query Set 2 Method 1 results with masks:
 k=1 -> 0.4 
 k=5 -> 0.45000000000000007 




## Method 2: RGB, Hellinger Kernel, 8 bins

In [22]:
BBDD_PATH = './BBDD/'
QSD2_PATH = './qsd2_w1/'

bbdd = get_dict_from_data(BBDD_PATH)
qsd2 = get_dict_from_data(QSD2_PATH)

histograms_bbdd_RGB = histogram(bbdd, color = "RGB")

all_results, dataframe, masks = get_masks_and_comparisons(qsd2)

dataframe_mean = dataframe.mean(axis=1)
print(f"{dataframe_mean} \n\n")

masked_histograms_RGB = histogram(qsd2, mask = masks, color="RGB")

MAPK_QS2_RGB_1_mask = calculateMAPK(masked_histograms_RGB, histograms_bbdd_RGB, "Hellinger_Kernel", cor2, 1, rev = True)
MAPK_QS2_RGB_5_mask = calculateMAPK(masked_histograms_RGB, histograms_bbdd_RGB, "Hellinger_Kernel", cor2, 5, rev = True)

print(f"Query Set 2 Method 2 results with masks:\n k=1 -> {MAPK_QS2_RGB_1_mask} \n k=5 -> {MAPK_QS2_RGB_5_mask} \n\n")

precision    0.836096
recall       0.916969
f1           0.868773
dtype: float64 


Query Set 2 Method 2 results with masks:
 k=1 -> 0.3 
 k=5 -> 0.3444444444444444 


